# SQLAlchemy 1: Basics

## TODO
Reflectievraag (klassikaal):
Waarom zou je niet altijd een ORM gebruiken?
(Hint: performance, specifieke SQL-functies, debugging)


## Inleiding
In deze cursus leer je wat een ORM is en hoe je SQLAlchemy kan gebruiken om op een betere en handigere manier met een SQL database te werken vanuit je Python applicaties.

De cursus volgt grotendeels de uitstekende [SQLAlchemy Unified Tutorial](https://docs.sqlalchemy.org/en/20/tutorial/index.html)

### Terugblik
Tijdens de Flask lessen hebben we een SQLite database gebruikt m.b.v. de `sqlite` package uit de Python standard library.

Deze aanpak was prima voor een klein project maar wordt snel onhandig bij grotere applicaties en/of bij verdere ontwikkeling.
- We werken rechtstreeks met SQL-strings
- We moeten queries aanpassen als de database verandert
- We moeten zelf fouten afhandelen
- Als we ooit willen overschakelen naar een andere database moeten we heel wat code volledig herzien

### Wat is een ORM
Een **ORM** (_Object Relational Mapper_) vertaalt (_mapt_) automatisch tussen objecten in een programmeeromgeving en tabellen in een relationele databank.

Met een ORM schrijf je dus minder ruwe SQL en werk je meer op het niveau van Python objecten. Je koppelt hiermee de business logic in de applicatie los van de onderliggende database. We spreken some over "_persistence ignorance"_.

Het `INSERT` statement uit het laatste voorbeeld zou je dan ongeveer zo herschrijven:

```python
session.add(Movie(title="Monty Python and the Holy Grail"))
session.commit()
movie = session.execute(select(Movie)).first()
```

`movie` zal dan een 'echt' `Movie` _object_ zijn waarvan we attributen, zoals `movie.title`, kunnen gebruiken!

ORMs zijn doorgaans ook database-agnostisch wat betekent dat je met geen of weinig aanpassingen in je code de onderliggende database kan veranderen!

ORMs zijn zeker niet uniek voor Python. Zo is er Hibernate voor Java of ODB voor C++.

### SQLAlchemy

SQLAlchemy is dus een ORM (en eigenlijk _de_ ORM voor Python), maar kan ook gebruikt worden louter als abstractielaag tussen applicatie en database.

SQLAlchemy bestaat uit twee grote onderdelen"
- **CORE**
  - Engine: beheert de verbinding(en) met de database
  - SQL Expression Language: maakt het mogelijk SQL te bouwen met Python functies
  - Programmatisch definiëren van database schema
  - _Imperatieve_ stijl - je specificeert precies hoe de database er uitziet en hoe data verwerkt wordt
  - `import sqlalchemy`
- **ORM** - Object Relational Mapping
  - Mapt database rows and tables op Python objecten
  - Breidt de Core SQL Expression Language uit zodat SQL queries samengesteld en uitgevoerd kunnen worden op basis van deze objecten.
  - _Declaratieve_ stijl - je beschrijft het model laat SQLAlchemy de rest doen
  - `import sqlalchemy.orm` 

![](https://docs.sqlalchemy.org/en/20/_images/sqla_arch_small.png "Bron: https://docs.sqlalchemy.org/en/20/intro.html")

Tijdens deze cursus zullen we zowel SQLAlchemy Core als ORM bekijken.

### SQLite
Net als bij Flask starten we in eerste instantie met SQLite als onderliggende database.

SQLite is ideaal om mee te starten omdat er geen server installatie nodig is en alles in één enkel bestand wordt beheerd. We kunnen dan later ook bekijken hoe je dankzij SQLAlchemy gemakkelijk naar een andere database kan overstappen, zoals PostgreSQL.

Bovendien mag SQLite tegenwoordig zeker als een volwaardige database systeem aanzien worden. SQLite ondersteunt ACID-transacties en volgt het SQL-standaarden. Heel wat applicaties gebruiken SQLite standaard als database, bvb. iMessage, Google Chrome of Dropbox.

### Voorbereiding
#### SQLite CLI
De SQLite CLI is handig om handmatig te bekijken wat er precies in de database gebeurt. Als je deze notebook gebruikt via Google Colab or Binder kan je via de terminal sqlite als volgt installeren.

```bash
apt install sqlite3
```

Voor andere platformen kan je terecht op  https://sqlite.org/download.html.

#### Python Package

```
pip install sqlalchemy~=2.0

uv add sqlalchemy~=2.0

conda install sqlalchemy~=2.0
```

## De SQLAlchemy Engine
De [`engine`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Engine) is de centrale bron van alle verbindingen met een bepaalde database.

Er is typisch één global `Engine` object.

De engine definieert wat voor database gebruikt worden (het _dialect_), bvb. `postgresql` of `sqlite`, als ook de details van die database (de _connection string).

Om een `Engine` object de maken gebruik je de `sqlalchemy.create_engine` functie (een zogenaamde [factory functie](https://en.wikipedia.org/wiki/Factory_method_pattern)).

In het volgend voorbeeld maken we een engine voor een SQLite database in het locale bestand `sqlite.db`. Je ziet ook hoe je een engine zou maken voor een [SQLite in-memory database](https://sqlite.org/inmemorydb.html). Dit laatste kan erg handig zijn bij testen. We werken nu eerst verder met een gewoon bestand zodat we de database zelf met de CLI kunnen inspecteren. Het `echo=True` argument zorgt er voor dat SQLAlchemy alle operaties (SQL) met de database logt. Zo kunnen we precies zien wat er gebeurt.

In [1]:
from sqlalchemy import create_engine

engine_file = create_engine("sqlite:///sqlite.db", echo=True)

engine_inmem = create_engine("sqlite+pysqlite:///:memory:", echo=True)

engine = engine_file

Er gebeurt eigenlijk nog niets. De engine gebruikt _lazy initialization_, wat in dit geval betekent dat er pas een verbinding met de database wordt gemaakt als die echt nodig is.

In de volgende stap werken we verder hetzelfde `Engine` object en gebruiken we de `connect` methode om een [`Connection` object](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Connection) te krijgen. Van dat object gebruiken we de `execute` methode een rechtstreeks een standaard SQL commando uit te voeren. Door `connect` als _context manager_ te gebruiken zorgen we er voor dat de verbinding automatisch terug vrij wordt gegeven (anders zouden we explicit the `conn.close()` methode moeten uitvoeren.). Op de `text` functie komen we later terug.

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT 'hello world'"))
    print(result.all())

2025-10-11 19:08:37,922 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:08:37,923 INFO sqlalchemy.engine.Engine SELECT 'hello world'
2025-10-11 19:08:37,923 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ()
[('hello world',)]
2025-10-11 19:08:37,923 INFO sqlalchemy.engine.Engine ROLLBACK


We zien dat:
- De `all()` methode op het resultaat van de query een lijst met tuples teruggeeft. Elk tuple is één rij.
- Als de context manager verlaten wordt is het standaard gedrag om een rollback (`conn.rollback()`) uit te voeren. Dit is een verstandig en veilig gedrag en dwingt het expliciet gebruik van `conn.commit()` indien er wijzigingen zijn die gecommit moeten worden.

Het `Engine` object bevat een connection [`pool`](https://docs.sqlalchemy.org/en/20/core/pooling.html#sqlalchemy.pool.Pool). Hiermee houdt SQLAlchemy zelf automatisch verschillende database verbindingen bij die via `engine.connect` toegekend worden.

In [3]:
print(engine.pool.status())

with engine.connect() as conn:
    print(engine.pool.status())
    with engine.connect() as conn2:
        print(engine.pool.status())
    print(engine.pool.status())

Pool size: 5  Connections in pool: 1 Current Overflow: -4 Current Checked out connections: 0
Pool size: 5  Connections in pool: 0 Current Overflow: -4 Current Checked out connections: 1
Pool size: 5  Connections in pool: 0 Current Overflow: -3 Current Checked out connections: 2
Pool size: 5  Connections in pool: 1 Current Overflow: -3 Current Checked out connections: 1


## RAW SQL

Gaan we in principe niet gebruiken maar goed te weten dat dit kan.

### CREATE, INSERT

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2025-10-11 19:13:54,584 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:13:54,585 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-10-11 19:13:54,586 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ()
2025-10-11 19:13:54,587 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) table some_table already exists
[SQL: CREATE TABLE some_table (x int, y int)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Door de [`text`](https://docs.sqlalchemy.org/en/20/core/sqlelement.html#sqlalchemy.sql.expression.text) functie te gebruiken kunnen op een veilige en database-onafhankelijke manier variabelen (_bound parameters_) worden meegegeven.

De stijl van transactie heet _commit-as-you-go_ omdat we expliciet the `commit()` methode moeten aanroepen.

Daar tegenover staat de _begin-once_ stijl. Door een context manager met `engine.begin()` (i.p.v. `engine.connect()`) wordt er automatisch gecommit wanneer de context manager verlaten wordt.

In [5]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2025-10-11 19:18:20,335 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:18:20,336 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-10-11 19:18:20,336 INFO sqlalchemy.engine.Engine [generated in 0.00046s] [(6, 8), (9, 10)]
2025-10-11 19:18:20,337 INFO sqlalchemy.engine.Engine COMMIT


Optioneel: inspecteer de database met de `sqlite3` CLI:

```bash
sqlite3 sqlite.db
.tables
.schema some_table
select * from some_table;
```

### SELECT

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y from some_table"))
    print(type(result))
    for row in result:
        print(f"row is a {type(row)}, row[0]={row[0]}, row.y={row.y}")

2025-10-11 19:23:10,737 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:23:10,738 INFO sqlalchemy.engine.Engine SELECT x, y from some_table
2025-10-11 19:23:10,738 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()
<class 'sqlalchemy.engine.cursor.CursorResult'>
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=1, row.y=1
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=2, row.y=4
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=6, row.y=8
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=9, row.y=10
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=6, row.y=8
row is a <class 'sqlalchemy.engine.row.Row'>, row[0]=9, row.y=10
2025-10-11 19:23:10,739 INFO sqlalchemy.engine.Engine ROLLBACK


Een [`Row`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row) object is vergelijkbaar met `NamedTuple`

In [7]:
from typing import NamedTuple

class FakeRow(NamedTuple):
    x: int
    y: int

fake_row = FakeRow(42, 9000)
print(f"fake_row[0]={fake_row[0]}, fake_row.y={fake_row.y}")

fake_row[0]=42, fake_row.y=9000


### SQL Parameters

Gebruik **altijd** _bound parameters_. Zo niet, dan loops je risico op _SQL Injection_ en/of conversieproblemen. Door bound parameters te gebruiken zorgt SQLAlchemy automatisch voorr de juiste escaping, quoting, en binding.

In [14]:
with engine.connect() as conn:
    # Zonder bound parameters:
    untrusted_variable = "1 OR 1=1"
    result = conn.execute(text(f"SELECT y from some_table where x = {untrusted_variable}"))
    print(list(result))

    # Met bound parameters:
    result = conn.execute(text("SELECT y from some_table where x = :x"), {"x": untrusted_variable})
    print(list(result))

2025-10-11 19:41:07,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:41:07,865 INFO sqlalchemy.engine.Engine SELECT y from some_table where x = 1 OR 1=1
2025-10-11 19:41:07,865 INFO sqlalchemy.engine.Engine [cached since 412.6s ago] ()
[(1,), (4,), (8,), (10,), (8,), (10,)]
2025-10-11 19:41:07,866 INFO sqlalchemy.engine.Engine SELECT y from some_table where x = ?
2025-10-11 19:41:07,866 INFO sqlalchemy.engine.Engine [generated in 0.00044s] ('1 OR 1=1',)
[]
2025-10-11 19:41:07,867 INFO sqlalchemy.engine.Engine ROLLBACK


## Metadata

Wat is _metadata_?

Metadata is de interne representatie van de database­structuur binnen SQLAlchemy, gebruik makend van Python objecten.

Basis objecten `MetaData`, `Table` en `Column`.

+- equivalent van DDL in de database (Data Definition Language).

SQLAlchemy gebruikt deze metadata om SQL-statements te genereren, of je nu met Core of ORM werkt.

```mermaid
classDiagram
        MetaData "1" *-- "*" Table
        Table "1" *-- "*" Column
```

### Metadata met Core

- [`Table`](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.Table)
- [`Column`](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.Column)

In [15]:
from sqlalchemy import MetaData, Table, Column, Integer, String

metadata = MetaData()

user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

print(type(user_table.c))
print(repr(user_table.c.id))
print(user_table.primary_key)

<class 'sqlalchemy.sql.base.ReadOnlyColumnCollection'>
Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False)
PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))


Een tweede table met een foreign key constaint.

`ForeignKey` krijgt een `str` met de `table.column` referentie, niet een `Table` of `Column` object zelf! (`"user_account.id"` ipv `user_account.id`)

In [16]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

Met de voorgaande voorbeelden verkrijgen we een Python-representatie van de tables, nog zonder dat die fysiek bestaan in de database.

Met `metadata.create_all()` wordt voor alle informatie in de metadata aangepaste DDL uitgevoerd in de database.

In [17]:
metadata.create_all(engine)

2025-10-11 19:55:33,708 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:55:33,709 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-10-11 19:55:33,709 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:55:33,710 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-10-11 19:55:33,710 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:55:33,711 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-10-11 19:55:33,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:55:33,712 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-10-11 19:55:33,712 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:55:33,713 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-10-11 19:55:33,713 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2025-10-11 19:55:33,715 INFO sqlalchemy.engine.Engine 
C

`metadata.create_all` is _idempotent_, wat wil zeggen dat je het verschillende keren kan uitvoeren zonder fouten te krijgen.

In [14]:
metadata.create_all(engine)

2025-08-10 15:02:43,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,516 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-08-10 15:02:43,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:02:43,517 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-08-10 15:02:43,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:02:43,518 INFO sqlalchemy.engine.Engine COMMIT


Met `metadata.drop_all` worden de tables opnieuw verwijderd uit de database.Merk op dat dit in de juiste (omgekeerde) volgorde gebeurt.

In [18]:
metadata.drop_all(engine)

2025-10-11 19:57:37,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-11 19:57:37,127 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-10-11 19:57:37,128 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:57:37,128 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-10-11 19:57:37,129 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-11 19:57:37,129 INFO sqlalchemy.engine.Engine 
DROP TABLE address
2025-10-11 19:57:37,130 INFO sqlalchemy.engine.Engine [no key 0.00034s] ()
2025-10-11 19:57:37,131 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2025-10-11 19:57:37,131 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2025-10-11 19:57:37,132 INFO sqlalchemy.engine.Engine COMMIT


### Metdata met ORM

#### Mapped Class

Conversie (_mapping_) tussen _data_ uit een database en objecten in een OO taal zoals Python.

Net zoals bij Core wordt een `Metadata` object gebruikt waarin de verschillende `Table` objecten worden bijgehouden. Dit (enkel) `Metadata` object wordt automatisch aangemaakt door `sqlalchemy.orm.DeclarativeBase` te subclassen in een eigen `Base` class. Elke Mapped Class wordt dan een subclass van `Base` en deelt daarmee dezelfde `Metadata`.

In [1]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

class Base(DeclarativeBase):
    pass

print(Base.metadata)

class User(Base):
    """This Mapped Class represents a single user."""
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

MetaData()


NameError: name 'String' is not defined

**Belangrijk**:
- Met Core _gebruiken_ we de bestaande `sqlalchemy.Table` class en maken we direct objecten aan.
  - Elk object = 1 database table
  - _Imperatief_: we definiëren precies hoe de database structuur er uit moet zien
- Met ORM maken we onze eigen _nieuwe_ classes (mapped classes).
  - Elke class = 1 table
  - Elk object = 1 row. We hebben nog geen objecten gemaakt.
  - _Declaratief_: we definiëren wat voor objecten we willen en laten SQLAlchemy de overeenkomstige database structuur opstellen
 
Een `DeclarativeBase` subclass maakt automatisch een `Table` class aan (herinnering: ORM _gebruikt_ Core!):

In [ ]:
print(repr(User.__table__))

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>, nullable=False), Column('fullname', String(), table=<user_account>, nullable=False), schema=None)


In [ ]:
class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]

Net als bij ORM zal `metadata.create_all()` de database DDL genereren en uitvoeren. Via de `Base` class krijgen we toegang tot het `metadata` object.

In [ ]:
Base.metadata.create_all(engine)

2025-10-12 09:16:48,052 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-12 09:16:48,053 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-10-12 09:16:48,053 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-12 09:16:48,054 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-10-12 09:16:48,055 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-12 09:16:48,056 INFO sqlalchemy.engine.Engine COMMIT


## Data verwerken met Core
In dit hoofdstuk bekijken we kort hoe SQLAlchemy Core gebruikt kan worden om met de database te werken. Ook al zijn we uiteindelijk van plan ORM te gebruiken is het interessant ook te begrijpen hoe je rechtstreeks de onderliggende Core kan werken. Heel wat concepten komen bovendien terug in ORM.

Waar we met `Metadata` en de `Table` objecten een abstractie kregen van de DDL, zo krijgen we hier een abstractie van de DML (Data Manipulation Language).

### INSERT

In [25]:
from sqlalchemy import insert

"""
Ter herinnering, user_table is een object dat op een imperatieve manier
de 'user_table' tabel uit de database voorstelt:

user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)
"""

statement = insert(user_table).values({"name": "wwhite", "fullname": "Walter White"})
print(type(statement))
print(statement)

<class 'sqlalchemy.sql.dml.Insert'>
INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


Met de `sqlalchemy.insert` functie en een `Table` object kunnen we op een programmatische manier een `INSERT` commando opstellen.

Net als na het aanmaken van een `Table` object is er enkel door het aanmaken van een `Insert` object nog geen interactie met de database. Er is immers nog geen gebruik gemaakt van de `engine`.

Uitvoeren gebeurt op dezelfde manier als het uitvoeren van zelfgeschreven SQL.

In [26]:
with engine.connect() as conn:
    # Ter vergelijking, met 'raw SQL':
    # result = conn.execute(
    #     text("INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)"),
    #     {"name": "wwhite", "fullname": "Walter White"},
    # )

    statement = insert(user_table).values({"name": "wwhite", "fullname": "Walter White"})
    result = conn.execute(statement)
    conn.commit()

    print(result)
    print(result.inserted_primary_key)

2025-10-12 10:05:14,033 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-12 10:05:14,033 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-10-12 10:05:14,034 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ('wwhite', 'Walter White')
2025-10-12 10:05:14,036 INFO sqlalchemy.engine.Engine COMMIT
<class 'sqlalchemy.engine.cursor.CursorResult'>
(1,)


In [ ]:
statement = user_table.insert().values({"name": "jpinkman", "fullname": "Jesse Pinkman"})
with engine.begin() as conn:
    conn.execute(statement)

2025-08-10 15:02:43,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,603 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-08-10 15:02:43,604 INFO sqlalchemy.engine.Engine [cached since 0.007483s ago] ('jpinkman', 'Jesse Pinkman')
2025-08-10 15:02:43,604 INFO sqlalchemy.engine.Engine COMMIT


### SELECT
Op gelijkaardige wijze kan met de `sqlalchemy.select` functie een `SELECT` statement samengesteld worden.

In [29]:
from sqlalchemy import select

statement = select(user_table)
print(type(statement))
print(statement)

<class 'sqlalchemy.sql.selectable.Select'>
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


#### WHERE
De `select` functie geeft een `Select` object terug. Dit object heeft verschillende methodes waarmee de query verder verfijnd en aangepast kan worden, bijvoorbeeld `.where()`. Elke van deze functies geeft opnieuw een `Select` object terug. Op die manier kan de gewenste query samengesteld worden door verschillende functies aan elkaar te hangen (_chaining_).

In [30]:
from sqlalchemy import select

statement = select(user_table).where(user_table.c.name == "wwhite")
print(type(statement))

with engine.connect() as conn:
    result = conn.execute(statement)
    print(type(result))
    for row in result:
        print(type(row))
        print(row)

<class 'sqlalchemy.sql.selectable.Select'>
2025-10-12 10:21:51,686 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-12 10:21:51,687 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-10-12 10:21:51,687 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ('wwhite',)
<class 'sqlalchemy.engine.cursor.CursorResult'>
<class 'sqlalchemy.engine.row.Row'>
(1, 'wwhite', 'Walter White')
2025-10-12 10:21:51,689 INFO sqlalchemy.engine.Engine ROLLBACK


Merk op dat het resultaat van de `execute` functie hetzelfde is als bij raw SQL, een `CursorResult` met `Row` objecten.


Als we specifieke kolommen van een tabel willen selecteren kan het `.c` attribuut van het `Table` object gebruikt worden. Dit kan gezien worden als een NamedTuple met alle `Column` objecten.

In [31]:
print(user_table.c)

statement = select(user_table.c.fullname)
with engine.connect() as conn:
    result = conn.execute(statement)
    for row in result:
        print(row)

ReadOnlyColumnCollection(user_account.id, user_account.name, user_account.fullname)
2025-10-12 10:30:48,322 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-12 10:30:48,323 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account
2025-10-12 10:30:48,323 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
('Walter White',)
2025-10-12 10:30:48,324 INFO sqlalchemy.engine.Engine ROLLBACK


`AND`
- Impliciet door `where()` chaining
- Impliciet door extra argument(en) in `where()`
- Expliciet met `and_()`

In [28]:
from sqlalchemy import and_

statement1 = select(user_table).where(user_table.c.name == "wwhite").where(user_table.c.fullname == "Walter White")
print(statement1)

statement2 = select(user_table).where(user_table.c.name == "wwhite", user_table.c.fullname == "Walter White")
print(statement2)

statement3 = select(user_table).where(and_(user_table.c.name == "wwhite", user_table.c.fullname == "Walter White"))
print(statement3)

print(str(statement1) == str(statement2) == str(statement3))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1
True


`OR`

Bonus: [`all()`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.CursorResult.all) en [`scalars()`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.CursorResult.scalars)

In [29]:
from sqlalchemy import or_

statement = select(user_table.c.name).where(or_(and_(user_table.c.name == "wwhite", user_table.c.fullname == "Walter White"), user_table.c.name == "jpinkman"))
print(statement)
with engine.connect() as conn:
    result = conn.execute(statement)
    print(result.all())

    result = conn.execute(statement)
    print(result.scalars().all())

SELECT user_account.name 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1 OR user_account.name = :name_2
2025-08-10 15:02:43,621 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,622 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE user_account.name = ? AND user_account.fullname = ? OR user_account.name = ?
2025-08-10 15:02:43,622 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('wwhite', 'Walter White', 'jpinkman')
[('wwhite',), ('jpinkman',)]
2025-08-10 15:02:43,622 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE user_account.name = ? AND user_account.fullname = ? OR user_account.name = ?
2025-08-10 15:02:43,623 INFO sqlalchemy.engine.Engine [cached since 0.001535s ago] ('wwhite', 'Walter White', 'jpinkman')
['wwhite', 'jpinkman']
2025-08-10 15:02:43,623 INFO sqlalchemy.engine.Engine ROLLBACK


#### INSERT FROM SELECT

In [30]:
select_stmt = select(user_table.c.id, user_table.c.name + "@gmail.com")
insert_stmt = insert(address_table).from_select(["user_id", "email_address"], select_stmt)

with engine.begin() as conn:
    conn.execute(insert_stmt)

    select_stmt = select(user_table.c.id, user_table.c.name + "@amc.com")
    insert_stmt = insert(address_table).from_select(["user_id", "email_address"], select_stmt)
    conn.execute(insert_stmt)

2025-08-10 15:02:43,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,627 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account
2025-08-10 15:02:43,627 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ('@gmail.com',)
2025-08-10 15:02:43,628 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account
2025-08-10 15:02:43,628 INFO sqlalchemy.engine.Engine [cached since 0.001245s ago] ('@amc.com',)
2025-08-10 15:02:43,628 INFO sqlalchemy.engine.Engine COMMIT


#### FROM en JOIN

- FROM is impliciet voor alle tables gebruikt `select()` argumenten.
- Probleem met het volgende?

In [31]:
statement = select(user_table.c.name, address_table.c.email_address)

with engine.connect() as conn:
    result = conn.execute(statement)
    print(result.all())

2025-08-10 15:02:43,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,632 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address 
FROM user_account, address
2025-08-10 15:02:43,632 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ()
[('wwhite', 'wwhite@gmail.com'), ('wwhite', 'jpinkman@gmail.com'), ('wwhite', 'wwhite@amc.com'), ('wwhite', 'jpinkman@amc.com'), ('jpinkman', 'wwhite@gmail.com'), ('jpinkman', 'jpinkman@gmail.com'), ('jpinkman', 'wwhite@amc.com'), ('jpinkman', 'jpinkman@amc.com')]
2025-08-10 15:02:43,633 INFO sqlalchemy.engine.Engine ROLLBACK


/var/folders/rf/vfsrnd6n627_7m0qqw5mmhnm0000gq/T/ipykernel_38831/1629152539.py:4: SAWarning: SELECT statement has a cartesian product between FROM element(s) "user_account" and FROM element "address".  Apply join condition(s) between each element to resolve.
  result = conn.execute(statement)


Twee basis manieren om te joinen.

1. `join_from(left_table, right_table)`

-> `FROM left_table JOIN right_table ON left_table.id = right_table.ref_id` 

In [32]:
statement = select(user_table.c.name, address_table.c.email_address).join_from(user_table, address_table)

with engine.connect() as conn:
    result = conn.execute(statement)
    print(result.all())

2025-08-10 15:02:43,635 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,636 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2025-08-10 15:02:43,636 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ()
[('wwhite', 'wwhite@gmail.com'), ('jpinkman', 'jpinkman@gmail.com'), ('wwhite', 'wwhite@amc.com'), ('jpinkman', 'jpinkman@amc.com')]
2025-08-10 15:02:43,636 INFO sqlalchemy.engine.Engine ROLLBACK


2. `join(right_table)`

- left table is afgeleid
- `ON` ook afgeleid, maar kan je expliciet meegeven in `join()`

In [33]:
print(select(user_table.c.name, address_table.c.email_address).join(address_table))
print(select(user_table.c.name, address_table.c.email_address).join(address_table, user_table.c.id == address_table.c.user_id))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Outer en Full joins.

Herinnering:

![](https://www.w3schools.com/sql/img_left_join.png) ![](https://www.w3schools.com/sql/img_full_outer_join.png)

(bron: https://www.w3schools.com/sql/sql_join.asp)

In [34]:
print(
    select(user_table.c.name, address_table.c.email_address).join(
        address_table, isouter=True
    )
)

print(
    select(user_table.c.name, address_table.c.email_address).join(
        address_table, full=True
    )
)

SELECT user_account.name, address.email_address 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.name, address.email_address 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


#### Functions

Bonus: `scalars()` en `one()`
- Let op: geen `.all()` op `result`!

In [35]:
from sqlalchemy import func

statement = select(func.count()).select_from(user_table)

with engine.connect() as conn:
    result = conn.execute(statement).scalars().one()
    print(result)

2025-08-10 15:02:43,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,645 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM user_account
2025-08-10 15:02:43,645 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ()
2
2025-08-10 15:02:43,646 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
statement = select(func.upper(user_table.c.name))

with engine.connect() as conn:
    result = conn.execute(statement).all()
    print(result)

2025-08-10 15:02:43,649 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,649 INFO sqlalchemy.engine.Engine SELECT upper(user_account.name) AS upper_1 
FROM user_account
2025-08-10 15:02:43,649 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
[('WWHITE',), ('JPINKMAN',)]
2025-08-10 15:02:43,650 INFO sqlalchemy.engine.Engine ROLLBACK


Let op!
- Ongedefinieerde functies worden letterlijk omgezet naar SQL

In [37]:
print(select(func.make_me_rich(user_table.c.name)))

SELECT make_me_rich(user_account.name) AS make_me_rich_1 
FROM user_account


#### ORDER BY, GROUP BY, HAVING

Bonus: `label()`

In [38]:
print(select(user_table).order_by(user_table.c.name))
print(
    select(user_table.c.name, func.count(address_table.c.id).label("count"))
    .join(address_table)
    .group_by(user_table.c.name)
    .having(func.count(address_table.c.id) > 1)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name
SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > :count_1


### UPDATE en DELETE

In [39]:
from sqlalchemy import update

statement = (
    update(user_table)
    .where(user_table.c.name == "wwhite")
    .values(fullname="Walter 'Heisenberg' White")
)

with engine.begin() as conn:
    conn.execute(statement)

2025-08-10 15:02:43,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,664 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2025-08-10 15:02:43,665 INFO sqlalchemy.engine.Engine [generated in 0.00032s] ("Walter 'Heisenberg' White", 'wwhite')
2025-08-10 15:02:43,665 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
from sqlalchemy import delete

statement = delete(user_table).where(user_table.c.name == "wwhite")

with engine.connect() as conn:
    result = conn.execute(statement)
    print(result.rowcount)

2025-08-10 15:02:43,669 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,669 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.name = ?
2025-08-10 15:02:43,669 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ('wwhite',)
1
2025-08-10 15:02:43,670 INFO sqlalchemy.engine.Engine ROLLBACK


## Data verwerken met ORM

Ter herinnering:
- Core: bestaande `sqlalchemy.Table` class.
  - Elk object = 1 table 
- ORM eigen _nieuwe_ classes (subclasses van `sqlalchemy.orm.Base`)
  - Elke class = 1 table
  - Elk object = 1 row

### Session

Met ORM gebruiken we een `sqlalchemy.orm.Session` i.p.v. een `sqlalchemy.Connection`
- `Session` _gebruikt_ `Connection`
- Identieke `execute()` methode, kan ook op dezelfde manier en met Core `Table` objecten gebruikt worden


In [41]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    print(session.connection())
    result = session.execute(select(user_table.c.name))
    print(result.all())

2025-08-10 15:02:43,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,673 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account
2025-08-10 15:02:43,673 INFO sqlalchemy.engine.Engine [generated in 0.00022s] ()
[('wwhite',), ('jpinkman',)]
2025-08-10 15:02:43,674 INFO sqlalchemy.engine.Engine ROLLBACK


### Relationschips

Uitdrukken van de relatie tussen (objecten van) mapped classes.


In [ ]:
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

    addresses: Mapped[list["Address"]] = relationship(back_populates="user")


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]

    user: Mapped[User] = relationship(back_populates="addresses")

In [ ]:
user = User(name="wwhite", fullname="Walter White")
print(repr(user.name))
print(repr(user.addresses))

'wwhite'
[]


In [ ]:
Base.metadata.create_all(engine)

2025-08-10 15:02:43,589 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,590 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-08-10 15:02:43,590 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:02:43,590 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-08-10 15:02:43,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:02:43,591 INFO sqlalchemy.engine.Engine COMMIT


Verandert niets aan de table DDL.


ORM specifiek:

`Session` ...
- Beheert alle interacties met de database
- Tijdelijke opslagplaats voor ORM-objecten (_Identity Map_)
- Houdt bij welke objecten geladen of gewijzigd zijn
- Synchroniseert wijzigingen met de database (unit-of-work pattern)

### SELECT

- Gebruikt dezelfde 'SQL Expression Language' als Core
  - `sqlalchemy.select`


Maar nu krijgen we ORM objecten terug!

In [42]:
with Session(engine) as session:
    result = session.execute(select(user_table)).first()
    print(result)

    result = session.execute(select(User)).first()
    print(result)
    print(result[0].name)

2025-08-10 15:02:43,677 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,677 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:02:43,677 INFO sqlalchemy.engine.Engine [generated in 0.00022s] ()
(1, 'wwhite', "Walter 'Heisenberg' White")
2025-08-10 15:02:43,678 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:02:43,678 INFO sqlalchemy.engine.Engine [generated in 0.00021s] ()
(<__main__.User object at 0x10e9f6e90>,)
wwhite
2025-08-10 15:02:43,679 INFO sqlalchemy.engine.Engine ROLLBACK


Of nog ...

Wat valt er op?

In [ ]:
with Session(engine) as session:
    result = session.execute(select(User)).scalars().first()
    print(result.id, result.name)

    result = session.scalars(select(User)).first()
    print(result.id, result.name)

    result = session.get(User, 1)
    print(result.id, result.name)

    result = session.scalars(select(User).where(User.name == "wwhite")).one()
    print(result.id, result.name)

2025-08-10 15:02:43,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,683 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:02:43,684 INFO sqlalchemy.engine.Engine [cached since 0.005759s ago] ()
1 wwhite
2025-08-10 15:02:43,685 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:02:43,685 INFO sqlalchemy.engine.Engine [cached since 0.006907s ago] ()
1 wwhite
1 wwhite
2025-08-10 15:02:43,686 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-08-10 15:02:43,686 INFO sqlalchemy.engine.Engine [generated in 0.00023s] ('wwhite',)
1 wwhite
2025-08-10 15:02:43,687 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
with Session(engine) as session:
    walter = session.get(User, 1)
    print(walter.fullname)
    print(walter.addresses)

2025-08-10 15:02:43,689 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,692 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-08-10 15:02:43,692 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1,)
Walter 'Heisenberg' White
2025-08-10 15:02:43,695 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2025-08-10 15:02:43,695 INFO sqlalchemy.engine.Engine [generated in 0.00028s] (1,)
[<__main__.Address object at 0x10e9fcec0>, <__main__.Address object at 0x10eafc190>]
2025-08-10 15:02:43,696 INFO sqlalchemy.engine.Engine ROLLBACK


### INSERT (add)

"Unit-of-Work" pattern
- Mapped class objecten (=~ rows) worden toegevoegd (get, add) aan een `Session`
- De `Session` houdt aanpassingen aan de objecten bij
- Die worden periodiek naar de database gestuurd (flush)

In [45]:
gus = User(name="gfring", fullname="Gus Fring")
print(gus)

In [46]:
def user_repr(self) -> str:
    return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

User.__repr__ = user_repr

print(gus)

User(id=None, name='gfring', fullname='Gus Fring')


In [47]:
adr_1 = Address(email_address="gus.fring@hermanos.com")
gus.addresses.append(adr_1)

print(gus)

with Session(engine) as session:
    session.add(gus)
    session.flush()
    print(gus)

User(id=None, name='gfring', fullname='Gus Fring')
2025-08-10 15:02:43,704 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,704 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-08-10 15:02:43,705 INFO sqlalchemy.engine.Engine [generated in 0.00026s] ('gfring', 'Gus Fring')
2025-08-10 15:02:43,706 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2025-08-10 15:02:43,706 INFO sqlalchemy.engine.Engine [generated in 0.00025s] (3, 'gus.fring@hermanos.com')
User(id=3, name='gfring', fullname='Gus Fring')
2025-08-10 15:02:43,706 INFO sqlalchemy.engine.Engine ROLLBACK


Rows toegevoegd in database?

Wat me de `id`'s in de objecten?

In [48]:
saul = User(name="sgoodman", fullname="Saul Goodman", addresses=[Address(email_address="bettercallsaul@goodman.com")])

with Session(engine) as session:
    session.add(saul)
    session.commit()

    print(saul)
    print(gus)

2025-08-10 15:02:43,710 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,710 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-08-10 15:02:43,711 INFO sqlalchemy.engine.Engine [cached since 0.006114s ago] ('sgoodman', 'Saul Goodman')
2025-08-10 15:02:43,711 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2025-08-10 15:02:43,711 INFO sqlalchemy.engine.Engine [cached since 0.005884s ago] (3, 'bettercallsaul@goodman.com')
2025-08-10 15:02:43,712 INFO sqlalchemy.engine.Engine COMMIT
2025-08-10 15:02:43,713 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,714 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-08-10 15:02:43,714 INFO sqlalchemy.engine.Engine [generated in 0.00029s] (3,)
User(id=3, name='sgoodman', full

Let op!
  - Detached Instance
  - `id` bekend

In [49]:
with Session(engine) as session:
    session.add(gus)
    session.commit()
    print(gus)

2025-08-10 15:02:43,718 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,719 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2025-08-10 15:02:43,719 INFO sqlalchemy.engine.Engine [cached since 0.005231s ago] (3,)
User(id=3, name='sgoodman', fullname='Saul Goodman')
2025-08-10 15:02:43,720 INFO sqlalchemy.engine.Engine ROLLBACK


In [50]:
gus = User(name="gfring", fullname="Gus Fring", addresses=[Address(email_address="gus.fring@hermanos.com")])

adr_2 = Address(email_address="gus.fring@amc.com", user=gus)

print(gus)

with Session(engine) as session:
    session.add(gus)
    session.commit()
    print(gus)

User(id=None, name='gfring', fullname='Gus Fring')
2025-08-10 15:02:43,723 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,724 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-08-10 15:02:43,724 INFO sqlalchemy.engine.Engine [cached since 0.01946s ago] ('gfring', 'Gus Fring')
2025-08-10 15:02:43,725 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?) RETURNING id
2025-08-10 15:02:43,725 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/2 (ordered; batch not supported)] (4, 'gus.fring@hermanos.com')
2025-08-10 15:02:43,726 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?) RETURNING id
2025-08-10 15:02:43,726 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] (4, 'gus.fring@amc.com')
2025-08-10 15:02:43,726 INFO sqlalchemy.engine.Engine COMMIT
2025-08-10 15:02:43,727 INFO sqlalchemy.engine.Engine 

### UPDATE

- = object aanpassen en committen
- aangepast object is "dirty"

Bonus:
- `filter_by` ipv. `where`
- `scalar_one` ipv `scalars().one()`

In [51]:
with Session(engine) as session:
    gus = session.execute(select(User).filter_by(name="gfring")).scalar_one()

    print(gus in session)
    print(gus in session.dirty)

    gus.fullname = "Gustavo Fring"
    print(gus in session.dirty)

    session.commit()

2025-08-10 15:02:43,731 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,732 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-08-10 15:02:43,732 INFO sqlalchemy.engine.Engine [cached since 0.04632s ago] ('gfring',)
True
False
True
2025-08-10 15:02:43,733 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2025-08-10 15:02:43,734 INFO sqlalchemy.engine.Engine [generated in 0.00024s] ('Gustavo Fring', 4)
2025-08-10 15:02:43,734 INFO sqlalchemy.engine.Engine COMMIT


### DELETE

In [52]:
with Session(engine) as session:
    bad_addr = session.execute(select(Address).filter_by(email_address="gus.fring@amc.com")).scalar_one()
    session.delete(bad_addr)
    session.commit()

2025-08-10 15:02:43,737 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:43,738 INFO sqlalchemy.engine.Engine SELECT address.id, address.user_id, address.email_address 
FROM address 
WHERE address.email_address = ?
2025-08-10 15:02:43,738 INFO sqlalchemy.engine.Engine [generated in 0.00025s] ('gus.fring@amc.com',)
2025-08-10 15:02:43,739 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2025-08-10 15:02:43,739 INFO sqlalchemy.engine.Engine [generated in 0.00024s] (7,)
2025-08-10 15:02:43,740 INFO sqlalchemy.engine.Engine COMMIT


### Extras

#### SQLAlchemy types en `inspect`

`User.addresses` is geen gewone `list`

In [53]:
user = User(name="wwhite", fullname="Walter White")
print(user.addresses)
print(type(user.addresses))

user.addresses.append("definitely not an Address object")

[]
<class 'sqlalchemy.orm.collections.InstrumentedList'>


AttributeError: 'str' object has no attribute '_sa_instance_state'

In [ ]:
from sqlalchemy import inspect

user = User(name="hschrader", fullname="Hank Schrader")

with Session(engine) as session:
    print(inspect(user))
    print(inspect(user).__dict__)
    session.add(user)

    print(inspect(user).__dict__)
    session.flush()

    print(f"XXXX {inspect(user).__dict__}")

    session.commit()
    print(f"XXXX {inspect(user).__dict__}")


    print(user.id)
    # lazy load
    print(user.addresses)

{'modified': True}
{'modified': True, '_orphaned_outside_of_session': False, 'insert_order': 1, 'session_id': 11, '_strong_obj': User(id=None, name='hschrader', fullname='Hank Schrader'), 'mapper': <Mapper at 0x10e9f5bd0; User>}
2025-08-10 15:02:44,171 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:44,171 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-08-10 15:02:44,172 INFO sqlalchemy.engine.Engine [cached since 0.4674s ago] ('hschrader', 'Hank Schrader')
XXXX {'modified': False, '_orphaned_outside_of_session': False, 'insert_order': 1, 'session_id': 11, '_strong_obj': None, 'mapper': <Mapper at 0x10e9f5bd0; User>, 'key': (<class '__main__.User'>, (5,), None), '_instance_dict': <weakref at 0x118964fe0; to 'sqlalchemy.orm.identity.WeakInstanceDict' at 0x118960e10>, 'expired': False}
2025-08-10 15:02:44,173 INFO sqlalchemy.engine.Engine COMMIT
XXXX {'modified': False, '_orphaned_outside_of_session': False, 'insert_order': 1, '

#### Dataclasses en SQLAlchemy

- Genereert automatisch `__init__`,  `__repr__`, ...
- Opslag van gestructureerde data
- Minder boilerplate-code zelf schrijven

In [55]:
from dataclasses import dataclass, field

@dataclass
class DC_User():
    name: str
    fullname: str
    addresses: list[str] = field(default_factory=list)

user = DC_User(name="wwhite", fullname="Walter White")
print(user)

DC_User(name='wwhite', fullname='Walter White', addresses=[])


Vergelijk gebruik in IDE (VSCode, PyCharm, ...) met SQLAlchemy mapped class

```python
from dataclasses import dataclass, field

from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship


@dataclass
class DC_User:
    name: str
    fullname: str

    addresses: list[str] = field(default_factory=list)
    id: int = 0


class Base(DeclarativeBase):
    pass


class SQLA_User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

    addresses: Mapped[list["Address"]] = relationship(back_populates="user")


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]

    user: Mapped[SQLA_User] = relationship(back_populates="addresses")


# dc_user = DC_User(name="wwhite")

dc_user = DC_User(name="wwhite", fullname="Walter White")
print(dc_user)

sqla_user = SQLA_User(name="wwhite")
print(sqla_user)
```

Oplossing: `sqlalchemy.orm.MappedAsDataclass`

```python
class Base(MappedAsDataclass, DeclarativeBase):
    pass
```

#### JOIN

In [56]:
with Session(engine) as session:
    statement = (
        select(Address.email_address)
        .select_from(User)
        .join(User.addresses)
        .where(User.name == "wwhite")
    )
    for row in session.execute(statement).scalars().all():
        print(row)

2025-08-10 15:02:44,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:02:44,183 INFO sqlalchemy.engine.Engine SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id 
WHERE user_account.name = ?
2025-08-10 15:02:44,183 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ('wwhite',)
wwhite@gmail.com
wwhite@amc.com
2025-08-10 15:02:44,184 INFO sqlalchemy.engine.Engine ROLLBACK


#### Lazy Loading en Loader Strategies

In [ ]:
with Session(engine) as session:
    jesse = session.execute(select(User).filter_by(name="jpinkman")).scalar_one()
    print(jesse.fullname)

    print("Other things happen...")

    print(f"{jesse.addresses[0].email_address} got lazily loaded")

2025-08-10 15:04:33,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:04:33,699 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-08-10 15:04:33,700 INFO sqlalchemy.engine.Engine [cached since 110s ago] ('jpinkman',)
Jesse Pinkman
Other things happen...
2025-08-10 15:04:33,701 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2025-08-10 15:04:33,701 INFO sqlalchemy.engine.Engine [cached since 110s ago] (2,)
jpinkman@gmail.com got lazily loaded
2025-08-10 15:04:33,701 INFO sqlalchemy.engine.Engine ROLLBACK


Kan leiden tot "N+1 Probleem"

In [61]:
with Session(engine) as session:
    users = session.execute(select(User)).scalars() # 1 query
    for user in users:
        print([a.email_address for a in user.addresses])  # N queries!

2025-08-10 15:12:57,515 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:12:57,516 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:12:57,516 INFO sqlalchemy.engine.Engine [cached since 613.8s ago] ()
2025-08-10 15:12:57,518 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2025-08-10 15:12:57,518 INFO sqlalchemy.engine.Engine [cached since 613.8s ago] (1,)
['wwhite@gmail.com', 'wwhite@amc.com']
2025-08-10 15:12:57,519 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2025-08-10 15:12:57,519 INFO sqlalchemy.engine.Engine [cached since 613.8s ago] (2,)
['jpinkman@gmail.com', 'jpinkman@amc.com']
2025-08-10 15:12:57,520 INFO sqlal

Oplossingen:
- join (maar geen toegang tot `addresses` via `User`)

In [76]:
with Session(engine) as session:
    users = session.execute(select(User, Address).join(User.addresses).order_by(User.id, Address.id))
    for user, address in users.all():
        print(f"{user.name} {address.email_address}")

2025-08-10 15:26:18,430 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:26:18,432 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, address.id AS id_1, address.user_id, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2025-08-10 15:26:18,433 INFO sqlalchemy.engine.Engine [cached since 117.8s ago] ()
wwhite wwhite@gmail.com
wwhite wwhite@amc.com
jpinkman jpinkman@gmail.com
jpinkman jpinkman@amc.com
sgoodman bettercallsaul@goodman.com
gfring gus.fring@hermanos.com
2025-08-10 15:26:18,434 INFO sqlalchemy.engine.Engine ROLLBACK


- `selectinload`

In [80]:
from sqlalchemy.orm import selectinload

with Session(engine) as session:
    users = session.execute(select(User).options(selectinload(User.addresses))).scalars()
    for user in users:
        print([a.email_address for a in user.addresses])

2025-08-10 15:30:19,630 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:30:19,632 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-08-10 15:30:19,632 INFO sqlalchemy.engine.Engine [generated in 0.00043s] ()
2025-08-10 15:30:19,634 INFO sqlalchemy.engine.Engine SELECT address.user_id AS address_user_id, address.id AS address_id, address.email_address AS address_email_address 
FROM address 
WHERE address.user_id IN (?, ?, ?, ?, ?)
2025-08-10 15:30:19,634 INFO sqlalchemy.engine.Engine [generated in 0.00038s] (1, 2, 3, 4, 5)
['wwhite@gmail.com', 'wwhite@amc.com']
['jpinkman@gmail.com', 'jpinkman@amc.com']
['bettercallsaul@goodman.com']
['gus.fring@hermanos.com']
[]
2025-08-10 15:30:19,635 INFO sqlalchemy.engine.Engine ROLLBACK


Kan ook permanent in `User` class:

```python
class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

    addresses: Mapped[list["Address"]] = relationship(back_populates="user", lazy="selectinloadˇ")
```

## Constraints en Indexes

In [85]:
from sqlalchemy import UniqueConstraint

engine = engine_inmem

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

    __table_args__ = (
        UniqueConstraint("name", name="uq_user_name"),
    )

Base.metadata.create_all(engine)

2025-08-10 15:59:19,676 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 15:59:19,676 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-08-10 15:59:19,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:59:19,677 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-08-10 15:59:19,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 15:59:19,678 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT uq_user_name UNIQUE (name)
)


2025-08-10 15:59:19,678 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2025-08-10 15:59:19,679 INFO sqlalchemy.engine.Engine COMMIT


In [86]:
u1 = User(name="wwhite", fullname="Walter White")
u2 = User(name="wwhite", fullname="Also Walter White")

with Session(engine) as session:
    session.add(u1)
    session.add(u2)
    session.flush()

2025-08-10 16:09:39,702 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 16:09:39,703 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-08-10 16:09:39,704 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('wwhite', 'Walter White')
2025-08-10 16:09:39,704 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-08-10 16:09:39,705 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('wwhite', 'Also Walter White')
2025-08-10 16:09:39,706 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: user_account.name
[SQL: INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id]
[parameters: ('wwhite', 'Also Walter White')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [88]:
from sqlalchemy import Index

Base.metadata.drop_all(engine)

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str]

    __table_args__ = (
        UniqueConstraint("name", name="uq_user_name"),
        Index("ix_user_name", "name")
    )

Base.metadata.create_all(engine)

2025-08-10 16:13:22,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 16:13:22,383 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-08-10 16:13:22,383 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 16:13:22,385 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2025-08-10 16:13:22,385 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2025-08-10 16:13:22,386 INFO sqlalchemy.engine.Engine COMMIT
2025-08-10 16:13:22,388 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-10 16:13:22,389 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-08-10 16:13:22,389 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 16:13:22,390 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-08-10 16:13:22,390 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-10 16:13:22,391 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR NOT NULL, 

## PostgreSQL ipv sqlite

```bash
conda install psycopg2

createdb syntra
```

In [90]:
engine = create_engine("postgresql://postgres@localhost/syntra", echo=True)

Base.metadata.create_all(engine)

2025-08-12 09:10:51,310 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-12 09:10:51,311 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-12 09:10:51,312 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-12 09:10:51,312 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-12 09:10:51,313 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-12 09:10:51,313 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-12 09:10:51,314 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-12 09:10:51,317 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [91]:
with Session(engine) as session:
    session.execute(
        insert(User),
        [
            {"name": "wwhite", "fullname": "Walter White"},
            {"name": "jpinkman", "fullname": "Jesse Pinkman"},
            {"name": "gfring", "fullname": "Gus Fring"},
            {"name": "hschrade", "fullname": "Hank Schrader"},
            {"name": "mehrmant", "fullname": "Mike Ehrmantraut"},
        ],
    )
    session.commit()

2025-08-12 09:16:45,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-12 09:16:45,662 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name__0)s, %(fullname__0)s), (%(name__1)s, %(fullname__1)s), (%(name__2)s, %(fullname__2)s), (%(name__3)s, %(fullname__3)s), (%(name__4)s, %(fullname__4)s)
2025-08-12 09:16:45,663 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/1 (unordered)] {'name__0': 'wwhite', 'fullname__0': 'Walter White', 'name__1': 'jpinkman', 'fullname__1': 'Jesse Pinkman', 'name__2': 'gfring', 'fullname__2': 'Gus Fring', 'name__3': 'hschrade', 'fullname__3': 'Hank Schrader', 'name__4': 'mehrmant', 'fullname__4': 'Mike Ehrmantraut'}
2025-08-12 09:16:45,666 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
with Session(engine) as session:
    walter = session.get(User, 1)
    print(walter.fullname)

2025-08-12 09:17:23,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-12 09:17:23,907 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = %(pk_1)s
2025-08-12 09:17:23,908 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {'pk_1': 1}
Walter White
2025-08-12 09:17:23,910 INFO sqlalchemy.engine.Engine ROLLBACK


## EXTRA

#### Dataclasses
